In [ ]:
# IMPORTS
import os
import urllib.request
import tarfile
from keras.preprocessing import sequence 
from keras.preprocessing.text import Tokenizer
import numpy as np

# regular expression
import re 
from keras.models import Sequential
from keras.layers import Dense,Flatten,Embedding,SimpleRNN,LSTM

In [ ]:
# File Operations
def read_files(filetype):
    path = "./aclImdb/"
    file_list=[]
    positive_path=path + filetype+"/pos/"
    for f in os.listdir(positive_path):
        file_list+=[positive_path+f]   
    negative_path=path + filetype+"/neg/"
    for f in os.listdir(negative_path):
        file_list+=[negative_path+f]      
    print('read',filetype, 'files:',len(file_list))      
    all_labels = ([1] * 12500 + [0] * 12500)    
    all_texts  = []
    for fi in file_list:
        with open(fi,encoding='utf8') as file_input:
            filelines = file_input.readlines()       
            all_texts += [rm_tags(filelines[0])]         
    return all_labels,all_texts

In [ ]:
def pre_prcess(txt):
  import string
  txt = txt.lower()
  txt = txt.replace("<br />"," ")
  txt = txt.strip()
  esc = string.punctuation.replace(".","")
  txt = re.sub("[%s]" % re.escape(esc),"",txt)
  txt = re.sub(r'\.+', ".", txt)
  return txt

In [ ]:
# PREPROCESS
def rm_tags(text):
    re_tag = re.compile(r'<[^>]+>')
    return re_tag.sub('',text)
    # return pre_prcess(text)

In [ ]:
# Get the Data For Training and testing
url="http://ai.stanford.edu/~amaas/data/sentiment/aclImdb_v1.tar.gz"
filepath="aclImdb_v1.tar.gz"
if not os.path.isfile(filepath):
    result=urllib.request.urlretrieve(url,filepath)
    print('downloaded:',result)
if not os.path.exists("aclImdb"):
    tfile = tarfile.open("aclImdb_v1.tar.gz", 'r:gz')
    result=tfile.extractall('.')
y_train, x_train = read_files('train')
y_test, x_test = read_files('test')
print(x_train[:2])

downloaded: ('aclImdb_v1.tar.gz', <http.client.HTTPMessage object at 0x7fe5ee4983d0>)
read train files: 25000
read test files: 25000
["This is one of those unfortunate films that suffered an even more sad, unfortunate death at the box office. I saw this film at a local art cinema,in revival form,shortly after it tanked in mainstream cinemas. It certainly deserves to be approached a second time (or even a third). Sandra B. takes it to the limit by doing spoken word & taking on some well known songs in this piece (her version of Hank William's 'I'm So Lonesome I Could Cry' could easily move you to tears). Maybe someday, audiences will be ready to take this film a bit more seriously (but not without some well placed laughs,too). The film moves at a brisk pace (thanks to some nice editing),so that some viewers will not find it stale & boring. Perhaps a revival is just down the pipeline.", "There seems to be only two types of reviews of this film on the net. Those who hate it and curse Ralp

In [ ]:
# Tokenize the input
token = Tokenizer(num_words=5000)
token.fit_on_texts(x_train)
x_train_seq = token.texts_to_sequences(x_train)
x_test_seq = token.texts_to_sequences(x_test)
 
x_train_v = sequence.pad_sequences(x_train_seq,maxlen=500)
x_test_v =  sequence.pad_sequences(x_test_seq,maxlen=500)

# convert labels to  array
print(type(y_train))
y_train = np.asarray(y_train).astype('int').reshape((-1,1))
y_test = np.asarray(y_test).astype('int').reshape((-1,1))
print(type(y_train),y_train.shape)


<class 'list'>
<class 'numpy.ndarray'> (25000, 1)


In [ ]:
# Define, Compile Model
# This one is simple
# dense_model = Sequential()
# dense_model.add(Embedding(input_dim=5000,output_dim=50,input_length=500))
# dense_model.add(Flatten())
# #model.add(SimpleRNN(units=32))
# dense_model.add(Dense(units=256,activation='relu'))
# dense_model.add(Dense(units=1,activation='sigmoid'))
# dense_model.summary()
# dense_model.compile(loss='binary_crossentropy',optimizer='adam',metrics=['accuracy'])

In [ ]:
  # THis one is trined using LSTM Layer
  model = Sequential()
  model.add(Embedding(input_dim=5000,output_dim=100,input_length=500))
  # model.add(Flatten())
  # model.add(SpatialDropout1D(0.4))
  model.add(LSTM(500, dropout=0.2,return_sequences=True))#, recurrent_dropout=0.2))
  model.add(LSTM(50,return_sequences=True))#, recurrent_dropout=0.2))
  model.add(LSTM(25))#, recurrent_dropout=0.2))
  model.add(Dense(units=256,activation='relu'))
  model.add(Dense(1,activation='sigmoid'))
  model.summary()
  model.compile(loss='binary_crossentropy',optimizer='adam',metrics=['accuracy'])

Model: "sequential"
_________________________________________________________________
Layer (type)                 Output Shape              Param #   
embedding (Embedding)        (None, 500, 100)          500000    
_________________________________________________________________
lstm (LSTM)                  (None, 500, 500)          1202000   
_________________________________________________________________
lstm_1 (LSTM)                (None, 500, 50)           110200    
_________________________________________________________________
lstm_2 (LSTM)                (None, 25)                7600      
_________________________________________________________________
dense (Dense)                (None, 256)               6656      
_________________________________________________________________
dense_1 (Dense)              (None, 1)                 257       
Total params: 1,826,713
Trainable params: 1,826,713
Non-trainable params: 0
______________________________________________

In [ ]:
x = model.get_layer(index=1)

In [ ]:
x.get_config()

{'activation': 'tanh',
 'activity_regularizer': None,
 'bias_constraint': None,
 'bias_initializer': {'class_name': 'Zeros', 'config': {}},
 'bias_regularizer': None,
 'dropout': 0.2,
 'dtype': 'float32',
 'go_backwards': False,
 'implementation': 2,
 'kernel_constraint': None,
 'kernel_initializer': {'class_name': 'GlorotUniform',
  'config': {'seed': None}},
 'kernel_regularizer': None,
 'name': 'lstm',
 'recurrent_activation': 'sigmoid',
 'recurrent_constraint': None,
 'recurrent_dropout': 0.0,
 'recurrent_initializer': {'class_name': 'Orthogonal',
  'config': {'gain': 1.0, 'seed': None}},
 'recurrent_regularizer': None,
 'return_sequences': True,
 'return_state': False,
 'stateful': False,
 'time_major': False,
 'trainable': True,
 'unit_forget_bias': True,
 'units': 500,
 'unroll': False,
 'use_bias': True}

In [ ]:
# Train and evaluate the model
train_his = model.fit(x_train_v,y_train,batch_size=50,epochs=10,verbose=1,validation_split=0.1)
scores = model.evaluate(x_test_v,y_test,verbose=1)
scores[1]

Epoch 1/10
450/450 [==============================] - 132s 283ms/step - loss: 0.5100 - accuracy: 0.7326 - val_loss: 0.4851 - val_accuracy: 0.8060
Epoch 2/10
450/450 [==============================] - 125s 279ms/step - loss: 0.2711 - accuracy: 0.8949 - val_loss: 0.3022 - val_accuracy: 0.8836
Epoch 3/10
450/450 [==============================] - 125s 278ms/step - loss: 0.1949 - accuracy: 0.9266 - val_loss: 0.4863 - val_accuracy: 0.8236
Epoch 4/10
450/450 [==============================] - 125s 278ms/step - loss: 0.1331 - accuracy: 0.9548 - val_loss: 0.4195 - val_accuracy: 0.8600
Epoch 5/10
450/450 [==============================] - 125s 278ms/step - loss: 0.1057 - accuracy: 0.9654 - val_loss: 0.5907 - val_accuracy: 0.7888
Epoch 6/10
450/450 [==============================] - 125s 279ms/step - loss: 0.0740 - accuracy: 0.9776 - val_loss: 0.5238 - val_accuracy: 0.8324
Epoch 7/10
450/450 [==============================] - 126s 279ms/step - loss: 0.0529 - accuracy: 0.9844 - val_loss: 0.4373 -

0.8555200099945068

In [ ]:
# Train and evaluate the model
# train_his = dense_model.fit(x_train_v,y_train,batch_size=128,epochs=20,verbose=2,validation_split=0.1)
# scores = dense_model.evaluate(x_test_v,y_test,verbose=1)
# scores[1]

In [ ]:
model.predict(sequence.pad_sequences(token.texts_to_sequences(["the movie was best but food was worst"]),maxlen=1000))

array([[0.00216502]], dtype=float32)

In [ ]:
# dense_model.predict(sequence.pad_sequences(token.texts_to_sequences(["worst"]),maxlen=500))

In [ ]:
# # testing new google news trained embedding layer
# import tensorflow_hub as hub
# import tensorflow as tf

# model = Sequential()
# embedding = "https://tfhub.dev/google/nnlm-en-dim50/2"
# hub_layer = hub.KerasLayer(embedding, input_shape=[], dtype=tf.string, trainable=True)

# model = Sequential()
# model.add(hub_layer)
# model.add(tf.keras.layers.Dense(16, activation='relu'))
# model.add(tf.keras.layers.Dense(1))
# # model.build(input_shape=(None,3))# build the model by calling the build method
# model.build(np.array(x_train[:3]))
# model.summary()
# # # model.build(())
# # model.summary()
# # model.compile(loss='binary_crossentropy',optimizer='adam',metrics=['accuracy'])

In [ ]:
# # Train and evaluate the model

# x_train_np_arr = []
# for x in x_train:
#   x_train_np_arr.append(np.array([x]))
# x_test_np_arr = []
# for x in x_test:
#   x_test_np_arr.append(np.array([x]))
# print(len(x_train_np_arr),len(x_test_np_arr))
# train_his = model.fit(x_train,y_train,batch_size=50,epochs=10,verbose=2)
# scores = model.evaluate(x_test,y_test,verbose=1)
# scores[1]

In [ ]:
type(x_train)

In [ ]:
import pickle
import joblib

In [ ]:
filename = 'finalized_model.sav'
# pickle.dump(model, open(filename, 'wb'))
joblib.dump(model, filename)

TypeError: ignored

In [ ]:
# load the model from disk
loaded_model = joblib.load(open(filename, 'rb'))
result = loaded_model.score(X_test, Y_test)
print(result)

EOFError: ignored

In [ ]:
while True:pass

KeyboardInterrupt: ignored